In [2]:
# import all important imports
import json
from typing import *

In [3]:
# parse the data
def read_jsonl(file_path):
    parsed_data = []
    with open(file_path, 'r') as file:
        parsed_data = [entity for entity in file]
    return parsed_data

In [4]:
def get_dataset_as_string(dataset: list):
    return ' '.join(dataset)

In [9]:
import openai
import tiktoken

In [10]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens + 10 # plus 10, cause this the maximum number of additional tokens

In [7]:
dataset = read_jsonl("scraped_data.jsonl")

In [11]:
total_tokens = num_tokens_from_string(get_dataset_as_string(dataset))

In [12]:
total_tokens

800270

In [13]:
# gpt-4 testing

In [20]:
def get_credentials(file_path="../credentials.json"):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [21]:
credentials = get_credentials()

In [32]:
openai_api_key = credentials['openai_api_key']

In [30]:
message_templeate = f"""Consider every JSON object in the given document as seperate entity. Now find out which entitites should be updated based on the user query. Speicify the number of json entity. Consider first json entity as number 0 and so on. Also specify which text should be replaced with which one based on the user query. Output format:entities=[], replcace of=(), replace with=()
query = (We removed the ability to archive queries, and instead added the ability to completely delete them. Update all relevant knowledge).
document:{get_dataset_as_string(dataset)}"""
#message_templeate

In [33]:
message_templeate = f"""Consider every JSON object in the given document as seperate entity. Now find out which entitites should be updated based on the user query. Speicify the number of json entity. Consider first json entity as number 0 and so on. Also specify which text should be replaced with which one based on the user query. Output format:entities=[], replcace of=(), replace with=()
query = (We removed the ability to archive queries, and instead added the ability to completely delete them. Update all relevant knowledge).
document:{get_dataset_as_string(dataset)}"""

messages=[
        {"role": "system", "content": """You are a helpful assistant to find replacable content from document and update based on user query."""},
        {"role": "user", "content": message_templeate}
      ]

from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
      model="gpt-4-0125-preview",
      messages=messages)

print(f'Response from openai :: {response.choices[0].message.content}')

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-turbo-preview in organization org-qDok5ZAfzcR20Yv4b13J6Nuc on tokens per min (TPM): Limit 300000, Requested 508777. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}